In [1]:
import libs
libs.print_hello()

Hello my capivaras!


In [2]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds with jokes",
    },
    {
        "role": "user",
        "content": "How many helicopters can a human eat in one sitting?",
    },
]


# Test TinyLlama

In [4]:
import torch
from transformers import pipeline

In [ ]:
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
# <|system|>
# You are a friendly chatbot who always responds in the style of a pirate.</s>
# <|user|>
# How many helicopters can a human eat in one sitting?</s>
# <|assistant|>
# ...

# Test TinyLlama over LlamaCpp

In [3]:
import requests
import openai

In [4]:
response = requests.get(url='http://llm-server:8080/health')
response

<Response [200]>

In [5]:
# Not-streaming, chat completion, http client
response = requests.post(
    url='http://llm-server:8080/v1/chat/completions',
    headers={'Content-Type': 'application/json'},
    json={'messages': messages},
)
response.json()

{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': 'The number of helicopters a human can eat in one sitting is undefined. This is because there is no one with a perfectly balanced stomach to determine this value.',
    'role': 'assistant'}}],
 'created': 1723038299,
 'model': 'gpt-3.5-turbo-0613',
 'object': 'chat.completion',
 'usage': {'completion_tokens': 38, 'prompt_tokens': 63, 'total_tokens': 101},
 'id': 'chatcmpl-phJjN2Cq7CfXKZPjG7GAeOxM4qIPKow5'}

In [6]:
# Streaming, chat completion, http client
response = requests.post(
    url='http://llm-server:8080/v1/chat/completions',
    headers={'Content-Type': 'application/json'},
    json={'messages': messages, 'stream': True},
    stream=True,
)
for chunk in response:
    print(chunk[:25], end='')
    #print('---------')

b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'lFHAOMYUbRs99w1YX","model'b'data: {"choices":[{"finis'b'OlFHAOMYUbRs99w1YX","mode'b'data: {"choices":[{"finis'b'gOlFHAOMYUbRs99w1YX","mod'b'data: {"choices":[{"finis'b'MhzugOlFHAOMYUbRs99w1YX",'b'data: {"choices":[{"finis'b'hzugOlFHAOMYUbRs99w1YX","'b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'gOlFHAOMYUbRs99w1YX","mod'b'data: {"choices":[{"finis'b'OlFHAOMYUbRs99w1YX","mode'b'data: {"choices":[{"finis'b'lFHAOMYUbRs99w1YX","model'b'data: {"choices":[{"finis'b'OlFHAOMYUbRs99w1YX","mode'b'data: {"choices":[{"finis'b'gOlFHAOMYUbRs99w1YX","mod'b'data: {"choices":[{"finis'b'gOlFHAOMYUbRs99w1YX","mod'b'data: {"choices":[{"finis'b'FHAOMYUbRs99w1YX","model"'b'data: {"choices":[{"finis'b'9w1YX","model":"gp

In [7]:
client = openai.OpenAI(
    base_url="http://llm-server:8080/v1",
    api_key = "not-needed"
)

In [25]:
# Not-streaming, text completion, openai client
completion = client.completions.create(
    model="not-needed",
    prompt='The capital of france is',
    max_tokens = 5,
)

print(completion.content)

 Paris, the capital of


In [9]:
# Not-streaming, chat completion, openai client
completion = client.chat.completions.create(
    model="not-needed",
    messages=messages,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="I do not have access to current nutritional information or human body measurements. However, in general, one helicoptor is defined as a medium-sized helicopter, which typically has a diameter of around 6 meters, and can carry a payload of around 1,000 kg (2,206 lb). Therefore, assuming a standard payload of 1,000 kg, and a helicopter with a diameter of 6 meters, it is safe to assume that one human can easily consume one or two helicoptors in one sitting. However, this may vary depending on factors such as the individual's weight, height, and the type and size of the helicopter being consumed.", refusal=None, role='assistant', function_call=None, tool_calls=None)


In [24]:
# Streaming, text completion, openai client
completion = client.completions.create(
    model="not-needed",
    prompt='The capital of france is',
    max_tokens = 5,
    stream=True,
)
for chunk in completion:
    print(chunk.content, end='')

 the city of lyon

In [12]:
# Streaming, chat completion, openai client
completion = client.chat.completions.create(
    model="not-needed",
    messages=messages,
    stream=True,
)

for chunk in completion:
    #print(chunk)
    print(chunk.choices[0].delta.content or '', end='')
    #print("****************")

A human can eat approximately 10 to 15 helicopters in one sitting. Helicopters are the popular snacks in certain cultures.

# Test TinyLlama over LlamaCpp over proxy

In [13]:
requests.get('http://orchestrator-proxy:80/docs')

<Response [200]>

In [28]:
# Streaming, chat completion, http client
# Works, but return all at once
response = requests.post(
    url='http://orchestrator-proxy:80/v1/chat/completions',
    headers={'Content-Type': 'application/json', 'Authorization': 'not-needed'},
    json={'messages': messages, 'stream': True},
    stream=True,
)
for chunk in response:
    print(chunk[:25], end='')
    #print('---------')

b'data: {"choices":[{"finis'b'EJjhDs092khERplYaD","mode'b'data: {"choices":[{"finis'b'dS95EJjhDs092khERplYaD","'b'data: {"choices":[{"finis'b'EJjhDs092khERplYaD","mode'b'data: {"choices":[{"finis'b'5EJjhDs092khERplYaD","mod'b'data: {"choices":[{"finis'b'EJjhDs092khERplYaD","mode'b'data: {"choices":[{"finis'b'JjhDs092khERplYaD","model'b'data: {"choices":[{"finis'b'EJjhDs092khERplYaD","mode'b'data: {"choices":[{"finis'b'JjhDs092khERplYaD","model'b'data: {"choices":[{"finis'b'S95EJjhDs092khERplYaD","m'b'data: {"choices":[{"finis'b'5EJjhDs092khERplYaD","mod'b'data: {"choices":[{"finis'b'5EJjhDs092khERplYaD","mod'b'data: {"choices":[{"finis'b'EJjhDs092khERplYaD","mode'b'data: {"choices":[{"finis'b'5EJjhDs092khERplYaD","mod'b'data: {"choices":[{"finis'b'qdS95EJjhDs092khERplYaD",'b'data: {"choices":[{"finis'b'qdS95EJjhDs092khERplYaD",'b'data: {"choices":[{"finis'b'EJjhDs092khERplYaD","mode'b'data: {"choices":[{"finis'b'qdS95EJjhDs092khERplYaD",'b'data: {"choices":[{"finis'b'qdS95EJjhDs092khER

In [15]:
client = openai.OpenAI(
    base_url="http://orchestrator-proxy:80/v1",
    api_key = "not-needed"
)

In [26]:
# Not-streaming, text completion, openai client
completion = client.completions.create(
    model="not-needed",
    prompt='I will answer this with one single word: The capital of france is',
    max_tokens = 5,
)

print(completion.content)

 not in france,


In [17]:
# Not-streaming, chat completion, openai client
completion = client.chat.completions.create(
    model="not-needed",
    messages=messages,
)

print(completion.choices[0].message.content)

One human can consume approximately 20 helicopters (or 20,000 units) of helicoptical jelly in one sitting. The exact quantity depends on the human's appetite and the quantity of helicoptical jelly available.


In [27]:
# Streaming, text completion, openai client
# Works, but return all at once
completion = client.completions.create(
    model="not-needed",
    prompt='The capital of france is',
    max_tokens = 5,
    stream=True,
)
for chunk in completion:
    print(chunk.content, end='')

 the city of paris

In [19]:
# Streaming, chat completion, openai client
# Works, but return all at once
completion = client.chat.completions.create(
    model="not-needed",
    messages=messages,
    stream=True,
)

for chunk in completion:
    #print(chunk)
    print(chunk.choices[0].delta.content or '', end='')
    #print("****************")

I do not have access to accurate information about the specific number of helicopters that a human can consume in one sitting, as this information is usually provided by the manufacturer or restaurant. However, it is recommended to consume one helicoptor per serving (about 180-220g), which is equivalent to the standard serving size of around 1/2 cup (50g). So, if you consume 6 servings per day, you will have consumed around 300-360 helicoptors.